In [1]:
# %%
# =============================================================================
# 1. IMPORTACIONES Y CONFIGURACIÓN INICIAL
# =============================================================================
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import torch
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import imageio # Necesario para crear los GIFs
import shutil
from time import time
import json
from pathlib import Path

# Crear una carpeta limpia para los frames de las animaciones
if os.path.exists("frames"):
    shutil.rmtree("frames")
os.makedirs("frames")


# %%
# =============================================================================
# 2. DEFINICIÓN DE LAS FUNCIONES OBJETIVO (Sin cambios)
# =============================================================================

def torch_fun_1(x):
    g = torch.Generator().manual_seed(42)
    # Usamos números de punto flotante para que los gradientes funcionen
    noise = (torch.rand(2, generator=g) * (20 - 4)) + 4
    y = x + noise
    # La función de puntuación espera un valor escalar de PyTorch
    if isinstance(y, torch.Tensor):
        return torch.sum(y**2)
    return sum(y**2)

def torch_fun_2(x):
    g = torch.Generator().manual_seed(42)
    noise = (torch.rand(10, generator=g) * (20 - 4)) + 4
    y = x + noise
    if isinstance(y, torch.Tensor):
        return torch.sum(y**2)
    return sum(y**2)

def tf_fun_1(x):
    g = tf.random.Generator.from_seed(42)
    y = x + g.uniform(shape=(2,), minval=4, maxval=20, dtype=tf.float32)
    return tf.reduce_sum(y**2)

def tf_fun_2(x):
    g = tf.random.Generator.from_seed(42)
    y = x + g.uniform(shape=(10,), minval=4, maxval=20, dtype=tf.float32)
    return tf.reduce_sum(y**2)


# %%
# =============================================================================
# 3. FUNCIÓN AUXILIAR PARA CREAR LAS ANIMACIONES GIF
# =============================================================================

def create_animation(loss_history, title, output_filename, frame_skip=10):
    """
    Crea una animación GIF a partir de un historial de valores de pérdida.
    """
    print(f"\n[INFO] Creando animación para '{title}'...")
    filenames = []
    
    # Limpiar la carpeta de frames para cada nueva animación
    if os.path.exists("frames"):
        shutil.rmtree("frames")
    os.makedirs("frames")

    for i in range(len(loss_history)):
        if i % frame_skip != 0 and i != len(loss_history) - 1:
            continue

        plt.style.use('seaborn-v0_8-whitegrid')
        fig, ax = plt.subplots(figsize=(10, 6), dpi=90)
        
        ax.plot(loss_history[:i+1], color='#0077b6', linewidth=2.5)
        ax.set_title(f"{title}\nPaso de Optimización: {i}", fontsize=16)
        ax.set_xlabel("Paso de Optimización", fontsize=12)
        ax.set_ylabel("Valor de la Función Objetivo (Loss)", fontsize=12)
        
        if np.max(loss_history) / (np.min(loss_history) + 1e-9) > 100:
            ax.set_yscale('log')

        ax.set_xlim(0, len(loss_history))
        ax.set_ylim(min(loss_history) * 0.9, max(loss_history) * 1.1)

        filename = f"frames/frame_{len(filenames):04d}.png"
        filenames.append(filename)
        plt.savefig(filename)
        plt.close(fig)

    gif_path = f"{output_filename}.gif"
    with imageio.get_writer(gif_path, mode='I', duration=0.1) as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)

    print(f"[SUCCESS] Animación guardada como: {gif_path}")


# %%
# =============================================================================
# 4. FUNCIONES DE OPTIMIZACIÓN (Adaptadas para score_fun y GIFs)
# =============================================================================
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
tf.random.set_seed(SEED)
OPTIMIZATION_STEPS = 500
LEARNING_RATE = 0.1

def optimize_torch_fun1(f):
    x = torch.randn((2,), requires_grad=True, generator=torch.Generator().manual_seed(SEED))
    optimizer = torch.optim.Adam([x], lr=LEARNING_RATE)
    loss_history = []
    for _ in range(OPTIMIZATION_STEPS):
        optimizer.zero_grad()
        loss = f(x)
        loss_history.append(loss.item())
        loss.backward()
        optimizer.step()
    create_animation(loss_history, "PyTorch Function 1", "pytorch_fun_1_optimization")
    return x

def optimize_torch_fun2(f):
    x = torch.randn((10,), requires_grad=True, generator=torch.Generator().manual_seed(SEED))
    optimizer = torch.optim.Adam([x], lr=LEARNING_RATE)
    loss_history = []
    for _ in range(OPTIMIZATION_STEPS):
        optimizer.zero_grad()
        loss = f(x)
        loss_history.append(loss.item())
        loss.backward()
        optimizer.step()
    create_animation(loss_history, "PyTorch Function 2", "pytorch_fun_2_optimization")
    return x

def optimize_tf_fun1(f):
    x = tf.Variable(tf.random.normal((2,), seed=SEED))
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    loss_history = []
    for _ in range(OPTIMIZATION_STEPS):
        with tf.GradientTape() as tape:
            loss = f(x)
        loss_history.append(loss.numpy())
        grads = tape.gradient(loss, [x])
        optimizer.apply_gradients(zip(grads, [x]))
    create_animation(loss_history, "TensorFlow Function 1", "tensorflow_fun_1_optimization")
    return x

def optimize_tf_fun2(f):
    x = tf.Variable(tf.random.normal((10,), seed=SEED))
    optimizer = tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    loss_history = []
    for _ in range(OPTIMIZATION_STEPS):
        with tf.GradientTape() as tape:
            loss = f(x)
        loss_history.append(loss.numpy())
        grads = tape.gradient(loss, [x])
        optimizer.apply_gradients(zip(grads, [x]))
    create_animation(loss_history, "TensorFlow Function 2", "tensorflow_fun_2_optimization")
    return x


# %%
# =============================================================================
# 5. FUNCIÓN DE PUNTUACIÓN Y BUCLE DE EVALUACIÓN
# =============================================================================

def score_fun(fun, optimization_fun, optimal_score, minimal_score_for_points, timeout):
    try:
        t_start = time()
        params = optimization_fun(fun)
        runtime = time() - t_start
        if runtime > timeout:
            print(f"Optimizer used {runtime:.2f}s, but only {timeout}s are allowed")

        funval_tensor = fun(params)
        # Asegurarse de que funval sea un número de Python para los cálculos
        if isinstance(funval_tensor, torch.Tensor):
            funval = funval_tensor.item()
        elif isinstance(funval_tensor, tf.Tensor):
            funval = funval_tensor.numpy()
        else:
            funval = funval_tensor

        print(f" -> Valor final de la función: {funval:.4f}")

        if funval > minimal_score_for_points:
            return 0.0
        else:
            score = np.round(float((100 * (minimal_score_for_points - funval) / (minimal_score_for_points - optimal_score))), 1)
            return max(0.0, score) # Asegurar que la puntuación no sea negativa
            
    except Exception as e:
        print(f"Ocurrió un error durante la puntuación: {e}")
        raise

# %%
# =============================================================================
# 6. BLOQUE DE EJECUCIÓN PRINCIPAL
# =============================================================================
if __name__ == "__main__":
    leaderboard_scores = []
    scores = []
    
    tasks = [
        ("PyTorch Function 1", torch_fun_1, optimize_torch_fun1, 0, 100),
        ("PyTorch Function 2", torch_fun_2, optimize_torch_fun2, 0, 100),
        ("TensorFlow Function 1", tf_fun_1, optimize_tf_fun1, 0, 100),
        ("TensorFlow Function 2", tf_fun_2, optimize_tf_fun2, 0, 100)
    ]

    for name, obj_fun, optimizer_fun, least_possible_fun_val, fun_val_to_receive_points in tasks:
        print(f"\n{'='*60}\nPROCESANDO Y PUNTUANDO: {name}\n{'='*60}")
        score = score_fun(obj_fun, optimizer_fun, least_possible_fun_val, fun_val_to_receive_points, timeout=60)
        leaderboard_scores.append({"name": name, "value": score})
        scores.append(score)
        print(f" => Puntuación para '{name}': {score}")

    # Escribir resultados
    folder = Path("results")
    folder.mkdir(exist_ok=True, parents=True)
    with open(f'{folder}/results.json', 'w') as f:
        json.dump({
            "score": np.mean(scores),
            "stdout_visibility": "visible",
            "leaderboard": leaderboard_scores
        }, f, indent=4)
        
    # Limpiar la carpeta de frames temporales al final
    if os.path.exists("frames"):
        shutil.rmtree("frames")
    
    print("\n\nProceso completado. Se han generado 4 archivos GIF y un archivo 'results/results.json'.")


2025-06-14 18:45:10.192194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749944710.292586   45580 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749944710.321611   45580 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749944710.521401   45580 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749944710.521458   45580 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749944710.521460   45580 computation_placer.cc:177] computation placer alr


PROCESANDO Y PUNTUANDO: PyTorch Function 1

[INFO] Creando animación para 'PyTorch Function 1'...


/tmp/ipykernel_45580/1661044365.py:101: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


[SUCCESS] Animación guardada como: pytorch_fun_1_optimization.gif
 -> Valor final de la función: 0.0058
 => Puntuación para 'PyTorch Function 1': 100.0

PROCESANDO Y PUNTUANDO: PyTorch Function 2

[INFO] Creando animación para 'PyTorch Function 2'...


/tmp/ipykernel_45580/1661044365.py:101: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


...


[INFO] Creando animación para 'TensorFlow Function 1'...


/tmp/ipykernel_45580/1661044365.py:101: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


[SUCCESS] Animación guardada como: tensorflow_fun_1_optimization.gif
 -> Valor final de la función: 0.0005
 => Puntuación para 'TensorFlow Function 1': 100.0

PROCESANDO Y PUNTUANDO: TensorFlow Function 2

[INFO] Creando animación para 'TensorFlow Function 2'...


/tmp/ipykernel_45580/1661044365.py:101: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


[SUCCESS] Animación guardada como: tensorflow_fun_2_optimization.gif
 -> Valor final de la función: 0.0081
 => Puntuación para 'TensorFlow Function 2': 100.0


Proceso completado. Se han generado 4 archivos GIF y un archivo 'results/results.json'.
